In [1]:
!pip install mpi4py

     |████████████████████████████████| 1.4MB 2.8MB/s 
  Created wheel for mpi4py: filename=mpi4py-3.0.3-cp36-cp36m-linux_x86_64.whl size=2074421 sha256=2b810287b5ad0121237dcf448b51df9e02d30a99acb0e3c76981c0b69a9d19d7
  Stored in directory: /root/.cache/pip/wheels/18/e0/86/2b713dd512199096012ceca61429e12b960888de59818871d6
Successfully built mpi4py


In [10]:
%%writefile colab-mpi.py
# this line above is magic code this will take the code below and write it into a file called colab-mpi.py
import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
print("I am rank " + str(rank))

randNum = numpy.zeros(1)

if rank == 1: #rank is identification for node
        randNum = numpy.random.random_sample(1)
        print("Process", rank, "drew the number", randNum[0])
        comm.Send(randNum, dest=0) #sends it random number to the 0 processes 
        comm.Recv(randNum, source=MPI.ANY_SOURCE)
        print("Process", rank, "received the number", randNum[0])

        randNum *= 2
        comm.Send(randNum, dest=0) #each send needs a corosponding received
        
if rank == 0:
        print("Process", rank, "before receiving has the number", randNum[0])
        comm.Recv(randNum, source=MPI.ANY_SOURCE) #this receives the number from other rank
        print("Process", rank, "received the number", randNum[0])
        randNum *= 2
        comm.Send(randNum, dest=1)

        comm.Recv(randNum, source=MPI.ANY_SOURCE)
        print("Process", rank, "received the number times 2", randNum[0])

Overwriting colab-mpi.py


In [11]:

#this box runs our code that we just saved before and define number of processes i.e. -n 2 or -n 4
#send the code to both processes
#changing -n 2 to -n4 changes nothing unless changed in the code 
!mpiexec --allow-run-as-root -n 4 python colab-mpi.py


I am rank 0
I am rank 2
Process 0 before receiving has the number 0.0
I am rank 1
I am rank 3
Process 1 drew the number 0.8295942683103475
Process 0 received the number 0.8295942683103475
Process 1 received the number 1.659188536620695
Process 0 received the number times 2 3.31837707324139
